In [7]:
import os
from datetime import datetime
import time
import torch
import random
import torchvision
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.io import ImageReadMode
from torch.utils.data import DataLoader
from torchsummary import summary
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Define constants
image_size = 224
batch_size = 8
num_epochs = 2
learning_rate = 0.001
num_classes = 4
dropout = 0.5
start_time = time.time()

# Function to read directories and label images
def label_images_in_directories(main_directory):
    label_names = []
    image_files = []
    for directory in os.listdir(main_directory):
        sub_directory = os.path.join(main_directory, directory)
        if os.path.isdir(sub_directory):
            for filename in os.listdir(sub_directory):
                image_file = os.path.join(sub_directory, filename)
                if os.path.isfile(image_file) and filename.endswith(".jpg"):
                    label_names.append(directory)
                    image_files.append(image_file)

    image_tensors = [torchvision.io.read_image(image, mode=ImageReadMode.UNCHANGED).to(torch.float32)/255 for image in image_files]
    nr_of_images = len(image_tensors)

    return label_names, image_tensors

# Load and label the images in the training directory
train_dir = "./../../Project 3/data/Data_cleaning_step1_2/Train"
label_names, image_tensors = label_images_in_directories(train_dir)

# Load and label the images in the test directory
test_dir = "./../../Project 3/data/Data_cleaning_step1_2/Test"
label_names, image_tensors = label_images_in_directories(test_dir)

# Define the transformations for train data before entering the neural network
transform_train = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomCrop(size=image_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Define the transformations for test data before entering the neural network
transform_test = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the training dataset
train_dataset = ImageFolder(train_dir, transform=transform_train)

# Split the dataset into training, validation, and test sets
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

dataset_size = len(train_dataset)
train_size = int(train_ratio * dataset_size)
val_size = int(val_ratio * dataset_size)
test_size = dataset_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    train_dataset, [train_size, val_size, test_size])

# Create DataLoaders for managing the data batches
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self, num_classes, image_size_nn, dropout):
        super().__init__()
        self.CNN_Apple = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * image_size_nn ** 2, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
            nn.Dropout(dropout)
        )

    def forward(self, x): 
        x = self.CNN_Apple(x)
        x = x.view(x.size(0), -1)
        return self.fc_layers(x)

# Create an instance of the model
image_size_nn = int(image_size/4)
model = CNNModel(num_classes, image_size_nn, dropout)
model = model.to(device)

# Print model summary
summary(model.to(device), input_size=(3, image_size, image_size))

# Define the loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, patience=2)

# Training function
def train(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()
        
    accuracy = 100.0 * correct / total
    return running_loss / len(train_loader), accuracy

# Test function for validation and test sets
def test(model, criterion, loader, device):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

    accuracy = 100.0 * correct / total
    return running_loss / len(loader), accuracy

# Initialize the best loss variable with infinity
best_loss = float('inf')

# Training loop
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, criterion, optimizer, train_loader, device)
    test_loss, test_accuracy = test(model, criterion, test_loader, device)
    val_loss, val_accuracy = test(model, criterion, val_loader, device)

    scheduler.step(test_loss) 

    print(f'Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
    
    # save model 
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(model.state_dict(), 'best_model_6.pth')
        with open("model_apple_7_x.pth", 'wb') as f:
            torch.save(model.fc_layers, f)

# Set a time and date
now = datetime.now()
timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
print(timestamp)

duration = time.time() - start_time
duration_str = time.strftime("%H:%M:%S", time.gmtime(duration))
print(f"Total duration: {duration_str}")
        
data = {
    "Timestamp": timestamp,
    "Duration": duration_str,
    "Model type": "resnet18",
    "Dataset use": os.path.basename(train_dir),
    "Image Resize": str(image_size)+"*"+str(image_size),
    "Epochs": num_epochs,
    "Learning rate": learning_rate,
    "Batch size": batch_size,
    "Train Accuracy": f"{train_accuracy:.2f}",
    "Validation accuracy": f"{val_accuracy:.2f}",
    "Test Accuracy": f"{test_accuracy:.2f}",
}
# Check if the CSV file already exists
if os.path.isfile("model_data.csv"):
    existing_data = pd.read_csv("model_data.csv")
    new_data = pd.concat([existing_data, pd.DataFrame(data, index=[0])], ignore_index=True)
else:
    new_data = pd.DataFrame(data, index=[0])

# Save the updated DataFrame to CSV
new_data.to_csv("model_data.csv", index=False)


cpu
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             448
              ReLU-2         [-1, 16, 224, 224]               0
         MaxPool2d-3         [-1, 16, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]           4,640
              ReLU-5         [-1, 32, 112, 112]               0
         MaxPool2d-6           [-1, 32, 56, 56]               0
            Linear-7                  [-1, 128]      12,845,184
              ReLU-8                  [-1, 128]               0
            Linear-9                   [-1, 32]           4,128
             ReLU-10                   [-1, 32]               0
           Linear-11                    [-1, 4]             132
          Dropout-12                    [-1, 4]               0
Total params: 12,854,532
Trainable params: 12,854,532
Non-trainable params: 0
---------------------